
# DETECTION OF PUPIL         
   * Johnal Dsouza
   * Abdullah Abdullah
_______________________

The pupil of the eye is detected using OpenCV and Python.
Various morphological operation such as Color-Conversion, Contrast-Enhancement, 
Blurring, Canny-Edge-Detection and Morphological-Closing.
_______________________


In [2]:
import cv2
import numpy as np
import random as rng
import math
import time
import datetime
frame_rate = []
time_rate = []
video_path = 'output2jd.mp4'  # Replace with your video file path

cap = cv2.VideoCapture(video_path)
Total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
center_list = {} 
frame_no =1


while cap.isOpened():
    
    ret, frame = cap.read()
    
    if not ret:
        break
    
    start = time.perf_counter()
    
    ###################################
    ##### PREPROCESSING OF IMAGE ######
    ###################################
    
    roi = frame[0:350, 0:500]
    # Convert image to grayscale
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    # Blurring of image
    blur = cv2.medianBlur(gray,5)
    blurred = cv2.GaussianBlur(blur, (7,7), 1.5, 1.5, cv2.BORDER_REPLICATE)
    
    # Detect edges
    edges = cv2.Canny(blurred, threshold1=10, threshold2=50)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (1,1))
    edges = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)
    
    # Find Contours
    _contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_TC89_KCOS)
    _drawing = np.copy(roi)
    
    
    _contours_filtered = []
    
    ###########################################
    ####### FINDING CIRCULAR CONTOURS #########
    ###########################################
    
    for contour_index, contour in enumerate(_contours):
        convex_hull = cv2.convexHull(contour)
        area_hull = cv2.contourArea(convex_hull)
        if  area_hull>280:
            # print(area_hull)
            circumference_hull = cv2.arcLength(convex_hull, True)
            # print(circumference_hull)
            if circumference_hull <= 400:
                circularity_hull = (4 * np.pi * area_hull) / circumference_hull ** 2
                if  circularity_hull > 0.85:
                    # print(circularity_hull)
                    _contours_filtered.append(convex_hull)
    
    ###################################################################
    ####### SELECTING THE BEST CIRCULAR CONTOUR ie. PUPIL #############
    ###################################################################
    
    min_circularity = 1.5
    min_circularity_circle = None
    
    minEllipse = [cv2.fitEllipse(c) for c in _contours_filtered]
    
    for i, ellipse in enumerate(minEllipse):
        circumference = cv2.arcLength(_contours_filtered[i], True)
        circularity = circumference ** 2 / (4 * math.pi * cv2.contourArea(_contours_filtered[i]))

        if circularity < min_circularity:
            min_circularity = circularity
            min_circularity_circle = ellipse
            
    ################################      
    ###### DRAWING THE PUPIL  ###### 
    ################################ 
    
    if min_circularity_circle is not None:
        contour_points = cv2.ellipse2Poly((int(min_circularity_circle[0][0]), int(min_circularity_circle[0][1])),
                                          (int(min_circularity_circle[1][0] / 2), int(min_circularity_circle[1][1] / 2)),
                                          int(min_circularity_circle[2]), 0, 360, 1)
        m = cv2.moments(contour_points)
        if m['m00'] != 0:
            center = (int(m['m10'] / m['m00']), int(m['m01'] / m['m00']))
            # The center of the pupil is stored in center_list, which will be used for Analysis
            center_list.update({frame_no: center})
            cv2.circle(_drawing, center, 3, (0, 255, 0), -1)   
        try:
            cv2.ellipse(_drawing, box=min_circularity_circle, color=(0, 255, 0),thickness = 2)   
        except:
            pass

    
    end = time.perf_counter()
    dtime = (end-start)
    
    if dtime > 0:  
        fps = 1 / dtime
    else:
        fps = float('inf')  

    
    time_rate.append(dtime)
    frame_rate.append(fps)
    cv2.putText(_drawing,str(fps), (0, 300),cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
    
    #out.write(_drawing)
    # Display Output
    cv2.imshow('median',blur)
    cv2.imshow('gray2', blurred)
    cv2.imshow('edges', edges)
    cv2.imshow('circles', _drawing)
    
    
    frame_no += 1

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

In [4]:
print(Total_frames)

1580


In [7]:
avg = sum(frame_rate) / len(frame_rate)
print(avg)

368.4964053871027


In [8]:
avg = sum(time_rate)/len(time_rate)
print(avg)

0.0028229724048103406


In [9]:
print(len(center_list))

1255


------------------------------
# ANNOTATION OF DATASET
The video data is annotated of each frame, with a rectangular box is drawn for each frame.
The top left (x,y) position and the bottom right (x,y) position of the rectangle is saved along with its frame no.
in a text file.
------------------------------

In [3]:
import cv2

# Global variables
current_frame = None
rectangle_points = []
rectangles_data = []

# Mouse callback function
def draw_rectangle(event, x, y, flags, param):
    global rectangle_points

    if event == cv2.EVENT_LBUTTONDOWN:
        rectangle_points = [(x, y)]

    elif event == cv2.EVENT_LBUTTONUP:
        rectangle_points.append((x, y))
        rectangles_data.append(rectangle_points)
        cv2.rectangle(current_frame, rectangle_points[0], rectangle_points[1], (0, 255, 0), 2)
        cv2.imshow("Video", current_frame)


def process_frame(frame, frame_number):
    global current_frame

    # Create a copy of the frame
    current_frame = frame.copy()

    # Display the frame number on the screen
    cv2.putText(current_frame, f"Frame: {frame_number}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    # Display the frame
    cv2.imshow("Video", current_frame)

    # Set the mouse callback function
    cv2.setMouseCallback("Video", draw_rectangle)

    # Wait for key press
    key = cv2.waitKey(0) & 0xFF

    # Check if 'q' is pressed to quit
    if key == ord('q'):
        return False

    return True


# Open the video file
video_path = "output2jd.mp4"
cap = cv2.VideoCapture(video_path)

# Check if video file opened successfully
if not cap.isOpened():
    print("Error opening video file")
    exit()

frame_number = 0
while cap.isOpened():
    # Read the current frame
    ret, frame = cap.read()

    # Increment the frame number
    frame_number += 1

    # Check if frame is read successfully
    if not ret:
        break

    # Process the frame
    result = process_frame(frame, frame_number)

    # Check if processing is stopped
    if not result:
        break

# Release the video capture object and close windows
cap.release()
cv2.destroyAllWindows()

# Save the rectangle data to a file
output_file = "rectangle_abc_trial.txt"
with open(output_file, "w") as file:
    for i,rect in enumerate(rectangles_data):
        file.write(f"{i+1}:({rect[0][0]},{rect[0][1]});({rect[1][0]},{rect[1][1]})\n")

print(f"Rectangles data saved to {output_file}")

Rectangles data saved to rectangle_abc_trial.txt


#### Converting Text data into dictionary

In [10]:
data_file = "jd_rectangle.txt"
##### CONVERTING ANNOTATED DATA IN DICTIONARY
rectangles_dict = {}
with open(data_file, "r") as file:
    for line in file:
        # Split the line into frame number and coordinates
        frame_num, *coord_str = line.strip().split(":")
        
        # Convert frame number to integer
        frame_num = int(frame_num)
        
        coordinates = []
        for coord in coord_str:
            # print(coord)
            x1y1,x2y2 = coord.split(";")
            # split coordinates into its individual components
            x1,y1 = x1y1.strip("()").split(",")
            x2,y2 = x2y2.strip("()").split(",")
            
        rectangles_dict[frame_num] = [(int(x1), int(y1)) , (int(x2),int(y2))]

# Print the dictionary
print(rectangles_dict)



{1: [(263, 48), (370, 129)], 2: [(235, 69), (338, 149)], 3: [(243, 78), (334, 153)], 4: [(236, 86), (323, 162)], 5: [(230, 90), (326, 167)], 6: [(233, 89), (323, 164)], 7: [(231, 89), (325, 166)], 8: [(235, 90), (319, 166)], 9: [(227, 90), (319, 164)], 10: [(234, 86), (326, 161)], 11: [(247, 77), (341, 145)], 12: [(257, 59), (342, 142)], 13: [(253, 64), (330, 124)], 14: [(250, 58), (346, 139)], 15: [(252, 57), (336, 143)], 16: [(252, 59), (346, 136)], 17: [(241, 56), (340, 143)], 18: [(248, 55), (344, 146)], 19: [(249, 59), (338, 131)], 20: [(250, 58), (338, 133)], 21: [(257, 58), (334, 134)], 22: [(255, 58), (338, 135)], 23: [(252, 54), (336, 137)], 24: [(254, 57), (330, 125)], 25: [(273, 59), (349, 126)], 26: [(271, 58), (353, 131)], 27: [(275, 56), (349, 132)], 28: [(257, 51), (349, 135)], 29: [(276, 52), (343, 125)], 30: [(273, 54), (342, 128)], 31: [(275, 56), (337, 129)], 32: [(273, 53), (341, 125)], 33: [(275, 52), (348, 134)], 34: [(274, 53), (344, 127)], 35: [(271, 55), (349, 

--------------------- 
# PUPIL DETECTION ANALYSIS
The pupil center is checked if it lies in between the bounding rectangle.
if Yes, number 1 is appended to detection, else 0
Further calculation of Accuracy, Precision, Recall and F1 score is found out
-------------------------

In [11]:
detection = []
for key in center_list.keys():
    if key in rectangles_dict.keys():
        
        center_value = center_list[key]
        rectangle_values = rectangles_dict[key]
        x_min, y_min = rectangle_values[0]
        x_max, y_max = rectangle_values[1]
        
        # check if pupil center lies in bounding rectangle
        if x_min <= center_value[0] <= x_max and y_min <= center_value[1] <= y_max:
            detection.append(1)
            # print(f"Key {key}: Center value {center_value} is within the rectangle.")
        else:
            detection.append(0)
            # print(f"Key {key}: Center value {center_value} is outside the rectangle.")
    else:
        print(f"Key {key} is not present in rectangles_dict.")
# print(detection)

In [12]:
Total_no_frames = Total_frames
Correctly_detect_pupil = detection.count(1) # Correctly detected pupil frames
Incorrect_detect_pupil = detection.count(0) # Incorrectly detected pupil frames
Missed_detect = Total_no_frames- Correctly_detect_pupil-Incorrect_detect_pupil # No detection of pupil in frames
print(f"Total frames = {Total_no_frames}")
print(f"Correctly detected pupil frames = {Correctly_detect_pupil}")
print(f"Incorrectly detected pupil frames = {Incorrect_detect_pupil}")
print(f"No detection of pupil in frame = {Missed_detect}")

Total frames = 1580
Correctly detected pupil frames = 1213
Incorrectly detected pupil frames = 42
No detection of pupil in frame = 325


In [13]:
Accuracy = (Correctly_detect_pupil/Total_no_frames)*100
Incorrect_detect_rate = (Incorrect_detect_pupil/Total_no_frames)*100
Missed_detect_rate = (Missed_detect/Total_no_frames)*100
Precision = (Correctly_detect_pupil/(Correctly_detect_pupil+Incorrect_detect_pupil))*100
Recall = (Correctly_detect_pupil/(Correctly_detect_pupil+Missed_detect))*100
F1_score = 2*(Precision*Recall)/(Precision+Recall)

print(f"Accuracy: {Accuracy:.2f}%")
print(f"Incorrect Detection Rate: {Incorrect_detect_rate:.2f}%")
print(f"Missed Detection Rate: {Missed_detect_rate:.2f}%")
print(f"Precision: {Precision:.2f}%")
print(f"Recall: {Recall:.2f}%")
print(f"F1 Score: {F1_score:.2f}")

Accuracy: 76.77%
Incorrect Detection Rate: 2.66%
Missed Detection Rate: 20.57%
Precision: 96.65%
Recall: 78.87%
F1 Score: 86.86
